# Crawling to SQLite3

This code walks through how to crawl and save crawled data in sql database. In detail, for a given url,
1. this code navigates all webpages inside.
2. for each webpage, collect the following information: title of the page; description; the number of external links; page contents; canonical.
3. the info on each webpage is saved as a row in database.
-------------------------------------

## import

In [62]:
import sqlite3
import re
import requests
from datetime import date

from bs4 import BeautifulSoup

headers= {'User-agent': 'Mozilla/5.0'}
date = date.today()

## Connecting to SQLite

In [63]:
db= sqlite3.connect('site_crawl.db')
cursor= db.cursor()

### Create the master table to keep the list of basic urls

In [65]:
cursor.execute("""CREATE TABLE IF NOT EXISTS Crawled_site_list (
                    URLID INTEGER PRIMARY KEY AUTOINCREMENT, 
                    url varchar(255), 
                    date varchar(255),
                    number_webpages_crawled integer
                )""")

### Get a basic url and create its database

In [66]:
url = input("URL to crawl: ") # the exact url (incl. http or https and www. if any) should be typed.
if len(url) < 1:
    url = "http://charlieojackson.co.uk"
    
def get_db_name(url):
    """Takes a URL and strips it to use as a table name"""    
    url_clense = re.findall(r'(ht.*://)?(www\.)?(.*?)\.', url)
    url_clense = url_clense[0][-1].capitalize()
    return url_clense

URL to crawl:  https://www.sec.gov


In [67]:
db_name= get_db_name(url)
cursor.execute(f"""DROP TABLE IF EXISTS """ + db_name)

cursor.execute(f"""CREATE TABLE {db_name}
               (URLID INTEGER PRIMARY KEY AUTOINCREMENT,
               URL varchar(255),
               Title varchar(255),
               Description varchar(255),
               InternalLinks INTEGER,
               ExternalLinks INTEGER, 
               PageContents TEXT, 
               Canonical varchar(255), 
               Time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
               )
               """)
print(url)

https://www.sec.gov


In [68]:
all_urls = []
all_urls.append(url)

## Functions to extract data from HTML source code

In [69]:
def extract_content(soup):
    """Extract required data for crawled page"""
    title= soup.title.string
    try:
        description= soup.find("meta", {"name":"description"})['content']
    except:
        description="Null"
        
    try:
        canonical = soup.find('link', {'rel':'canonical'})['href']
    except:
        canonical = "Null"
        
    contents_dirty= soup.text
    contents=contents_dirty.replace("\n","")
    return (title, description, contents, canonical)

In [70]:
def extract_links(soup):
    """Extract links and link contents from page"""
    links_dirty= soup.find_all('a')
    for link in links_dirty:
        if str(link.get('href')).startswith(url)== True and link.get('href') not in all_urls:
            if '.jpg' in link.get('href') or '.png' in link.get('href'):
                continue
            else:
                all_urls.append(link.get('href'))
            
    return(len(links_dirty))

In [71]:
def insert_data(extracted_data):
    """Insert the crawled data into the database""" 
    url, title, description, contents, no_of_links, canonical= extracted_data

    cursor.execute("INSERT INTO " + db_name + " (URL, Title, Description, ExternalLinks, PageContents, Canonical) VALUES(?,?,?,?,?,?)",(url, title, description, no_of_links, contents, canonical))

    db.commit()

## Starting the web crawler

In [72]:
link_counter= 0
while link_counter < len(all_urls):
    try:
        print(str(link_counter) + " crawling: " + all_urls[link_counter])
        r = requests.get(all_urls[link_counter], headers= headers)
        if r.status_code== 200:
            html= r.text
            soup= BeautifulSoup(html, "html.parser")
            no_of_links=extract_links(soup)
            title, description, contents, canonical = extract_content(soup)
            extracted_data= (all_urls[link_counter], title, description, contents, no_of_links,canonical)
            insert_data(extracted_data)
        link_counter += 1
        
    except Exception as e:
        link_counter += 1
        print(str(e))
        
    if link_counter>=5:
        break

0 crawling: https://www.sec.gov
1 crawling: https://www.sec.gov/litigation.shtml
2 crawling: https://www.sec.gov/divisions/enforce/claims.htm
3 crawling: https://www.sec.gov/litigation/litreleases/2022/lr25298.htm
4 crawling: https://www.sec.gov/litigation/litreleases/2021/lr25297.htm


### Add the basic url to the master table

In [73]:
cursor.execute("""INSERT INTO Crawled_site_list (url, date, number_webpages_crawled) VALUES(?, ?, ?)""", (url, date, link_counter))
db.commit()

### Display the list of basic urls

In [74]:
cursor.execute("""SELECT * from  Crawled_site_list""")
cursor.fetchall()

[(1, 'https://www.cnbc.com', '2022-01-04', 5),
 (2, 'http://charlieojackson.co.uk', '2022-01-04', 5),
 (3, 'https://www.sec.gov', '2022-01-04', 5)]

In [75]:
cursor.close()
db.close()